Precision and recall
https://insidelearningmachines.com/precisionk_and_recallk/#:~:text=Precision%40k%20and%20Recall%40k%20are%20metrics%20used%20to%20evaluate,end%20user%20by%20the%20model.

In [43]:
# imports
import pandas as pd
import openai
import numpy as np
from typing import List
import ast

In [2]:
news = pd.read_csv("evaluation/hybrid.csv") 
news.head()

,ID,category,sub_category,content,collaborative_rec,content_rec,mean,two_one
0,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as ...,0.198320,0.159909,0.179114,0.172712
1,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 vict...,0.216016,0.061615,0.138816,0.113082
2,N10414,movies,movienews,"Robert Evans, 'Chinatown' Producer and Paramou...",0.246655,0.274041,0.260348,0.264912
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,0.256467,0.263025,0.259746,0.260839
4,N63302,lifestyle,lifestylebuzz,This Wedding Photo of a Canine Best Man Captur...,0.259769,0.281452,0.270611,0.274225


In [40]:
# Calculating the mean of 'content_rec' column
mean_content_rec = news['content_rec'].mean() 

print("Mean of 'content_rec' column:", mean_content_rec)

Mean of 'content_rec' column: 0.2501734171562433


In [46]:
# Sorting the 'content_rec' column in ascending order
sorted_content_rec = news['content_rec'].sort_values(ascending=True)

# Calculating the index for the value representing the maximum of 4% of the sorted data
index_4_percent = int(len(sorted_content_rec) * 0.04)  # 4% of the data

# Getting the value at the calculated index
value_at_4_percent = sorted_content_rec.iloc[index_4_percent]

print("Value at 4% of the sorted 'content_rec' data:", value_at_4_percent)


Value at 4% of the sorted 'content_rec' data: 0.185792921450654


In [22]:
data = pd.read_csv("MIND/behaviors_test.csv", sep = ';') 
data.head()

,0,1,2,3,4
0,0,U13000,11/11/2019 9:05:58 AM,N42782 N18445 N49749,N7482-1 N6379-0
1,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
2,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0


In [23]:
data = data.drop(['0', '2', '3'], axis=1)
data.head()

,1,4
0,U13000,N7482-1 N6379-0
1,U13740,N55689-1 N35729-0
2,U91836,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,U73700,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,U34670,N35729-0 N33632-0 N49685-1 N27581-0


In [24]:
data.columns =['User', 'Impressions']
data.head()

,User,Impressions
0,U13000,N7482-1 N6379-0
1,U13740,N55689-1 N35729-0
2,U91836,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,U73700,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,U34670,N35729-0 N33632-0 N49685-1 N27581-0


In [25]:
data

,User,Impressions
0,U13000,N7482-1 N6379-0
1,U13740,N55689-1 N35729-0
2,U91836,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,U73700,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,U34670,N35729-0 N33632-0 N49685-1 N27581-0
...,...,...
95,U85027,N51048-0 N28072-1
96,U10585,N14592-0 N12029-0 N45704-0 N9806-0 N37497-0 N5...
97,U89839,N12042-0 N35729-0 N49685-0 N62729-1
98,U47761,N9734-0 N25949-0 N47061-0 N14726-0 N59272-0 N3...


In [33]:
# Function to count the number of suffixes
def count_suffixes(row, suffix):
    impressions = row['Impressions'].split()
    count = sum(1 for imp in impressions if imp.endswith(suffix))
    return count

# Counting "-1" and "-0" suffixes
data['-1 Count'] = data.apply(lambda row: count_suffixes(row, '-1'), axis=1)
data['-0 Count'] = data.apply(lambda row: count_suffixes(row, '-0'), axis=1)

# Total count across all users
total_minus_1 = data['-1 Count'].sum()
total_minus_0 = data['-0 Count'].sum()

print("Total -1 count:", total_minus_1)
print("Total -0 count:", total_minus_0)

Total -1 count: 137
Total -0 count: 3670


In [41]:
total_minus_1/total_minus_0

0.03732970027247957

In [26]:
def create_user_df(input_df, user):
    user_row = input_df[input_df['User'] == user]

    if user_row.empty:
        return None

    impressions = user_row['Impressions'].values[0].split()

    news_ids = []
    true_values = []

    for impression in impressions:
        news_id, true_value = impression.split('-')
        news_ids.append(news_id)
        true_values.append(int(true_value))

    user_df = pd.DataFrame({'ID': news_ids, 'true_value': true_values})
    return user_df

In [34]:
user_input = 'U13000'
user_df = create_user_df(data, user_input)
print(user_df)


      ID  true_value
0  N7482           1
1  N6379           0


In [35]:
# Create a dictionary mapping IDs to new column values from df2
id_to_new_column = news.set_index('ID')['content_rec'].to_dict()

# Add the new column to df1 based on the mapping
user_df['content_rec'] = user_df['ID'].map(id_to_new_column)

print(user_df)

      ID  true_value  content_rec
0  N7482           1     0.224894
1  N6379           0     0.261450


In [10]:
###precision###

In [11]:
###recall###

In [12]:
###F1###

Intra-list Similarity and others https://github.com/statisticianinstilettos/recmetrics